## Google Photos API - Download images from Google Photos using Python

Using the Google Photos REST API you can download, upload and modify images stored in Google Photos.

The following steps describe how to set up a simple project that lets you use Python to download images from Google Photos:

In [5]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install -r "requirements.txt"
!pip freeze > requirements.txt

In [2]:
!which python
!which pip

/Users/andyflury/opt/anaconda3/bin/python
/Users/andyflury/opt/anaconda3/bin/pip


## Use the Google Photo Library API for the first time:

The following section shows how to use OAuth Credentials for authentication with the Google Library API. The code section below covers the following steps:

8. Create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](read_me_img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [2]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


## Initialize photos api and create service

In [32]:
GOOGLE_PHOTOS_API = GooglePhotosApi()
CREDS = GOOGLE_PHOTOS_API.run_local_server()
def init_gp_server():
    global GOOGLE_PHOTOS_API, CREDS
    GOOGLE_PHOTOS_API = GooglePhotosApi()
    CREDS = GOOGLE_PHOTOS_API.run_local_server()

init_gp_server()

## Use pythons requests module and the token file to retrieve data from Google Photos

The functions here
1. request media items from Hilledwight shared album and store the item IDs in picIds.txt
2. request media items from all my photos organized by month and store the items IDs respective txt files in the idFiles directory

We're limiting the number of pictures requested by numPics in each function, but in the future we'll remove numPics

In [27]:
import json
import requests

PHOTOS_HEADERS = {
    'content-type': 'application/json',
    'Authorization': 'Bearer {}'.format(CREDS.token)
}

GET_LIBRARIES_URL = 'https://photoslibrary.googleapis.com/v1/albums'
MEDIA_ITEMS_URL = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'

MAX_NUM_PICS = 260
MONTHS = [
    {'month': 8, 'year':2021, 'name':'August'}, 
    {'month': 9, 'year':2021, 'name': 'September'}, 
    {'month': 10, 'year':2021, 'name': 'October'}, 
    {'month': 11, 'year':2021, 'name': 'November'}, 
    {'month': 12, 'year':2021, 'name': 'December'}, 
    {'month': 1, 'year':2022, 'name': 'January'}, 
    {'month': 2, 'year':2022, 'name': 'February'}, 
    {'month': 3, 'year':2022, 'name': 'March'},
    {'month': 4, 'year':2022, 'name': 'April'}, 
    {'month': 5, 'year':2022, 'name': 'May'}, 
    {'month': 6, 'year':2022, 'name': 'June'}, 
]

CATEGORIES = [
    ['ANIMALS,PETS','ANIMALS'],
    ['FASHION'],
    ['LANDMARKS'],
    ['ARTS'],
    ['FLOWERS','GARDENS','LANDSCAPES', 'NATURE'],
    ['BIRTHDAYS'],
    ['FOOD'],
    ['NIGHT'],
    ['SELFIES'],
    ['CITYSCAPES', 'HOUSES', 'CITYSCAPES'],
    ['PEOPLE'],
    ['SPORT'],
    ['HOLIDAYS'],
    ['CRAFTS'],
    ['PERFORMANCES'],
    ['TRAVEL'],
    ['RECEIPTS','WEDDINGS','WHITEBOARDS','SCREENSHOTS','UTILITY','DOCUMENTS', 'MISC']
]

def writeToFile(f, mediaItems):
    for item in mediaItems:
        try:
            f.write('%s\n' %item['id'])
        except:
            print('!!!! WARNING: Write error !!!!')

def get_hd_people_pics():
    print('Downloading HD pics...')
    try:
        res = requests.request("GET", GET_LIBRARIES_URL, headers=PHOTOS_HEADERS)
        res.json()
        albumID = res.json()['albums'][1]['id']
    except:
        print('!!!! WARNING: Library request error !!!!') 
        print(res)
    
    payload = {
      "albumId": albumID,
      "pageSize": "5"
    }
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('idFiles/picIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            numPics += len(res['mediaItems'])
            payload = {
              "albumId": albumID,
              "pageSize": "100",
              "pageToken": res['nextPageToken'],
            }
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            
    f.close()
    print('Download complete!')
    return numPics

def get_month_pics(monthEntry):
    payload = {
      "filters": {
        "dateFilter": {
          "dates": [
            {
              "month": monthEntry['month'],
              "year": monthEntry['year']
            }
           ]
            }
          },
        "pageSize": "25"
      }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('idFiles/months/'+monthEntry['name']+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            payload = {
              "filters": {
                "dateFilter": {
                  "dates": [
                    {
                      "month": monthEntry['month'],
                      "year": monthEntry['year']
                    }
                   ]
                }
              },
              "pageSize": "25",
              "pageToken": res['nextPageToken']
            }
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            numPics += len(res['mediaItems'])
    f.close()
    return numPics

def get_all_month_pics():
    print('Downloading pics with monthly filter...')
    for monthEntry in MONTHS:
        get_month_pics(monthEntry)
    print('Download complete!')
        
        
def get_category_pics(category):
    categoryLabel = category[0]
    if len(category)>1:
        categoryLabel = category[-1]
        category = category[0:len(category)-1]
    payload = {
      "filters": {
        "contentFilter": {
          "includedContentCategories": category
        }
      },
      "pageSize": "25"
    }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    numPics = 0
    with open('idFiles/categories/'+categoryLabel+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            payload = {
              "filters": {
                "contentFilter": {
                  "includedContentCategories": category
                }
              },
              "pageSize": "25",
              "pageToken": res['nextPageToken']
            }
            
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            numPics += len(res['mediaItems'])
    f.close()
    return numPics
    
def get_all_category_pics():
    print('Downloading pics with category filter...')
    for category in CATEGORIES:
        get_category_pics(category)
    print('Download complete!')

In [ ]:
testID='AMyo5r0lZANxOJrBM7XH887PZfWTyK_x6LgX_n51XULlarUSqYiLm8g-L4xEEsX08zcy2HKmEDI8iMp09XTA0RCHYCR0MfxB7w'
res = requestIMG(test)
downloadIMG(res['baseUrl'], 'testIMG.jpg')

## CompreFace Request

To request the CompreFace api, we need to first request the media item from the Google Photos API, which validates the baseUrl for 60 minutes, then download the image and then send this file in our request.

In [46]:
"""
    Request image with imgID
    We can only download the image from baseURL if we have requested
    the url from GP API and make our download request within 60 minutes
"""

def requestIMG(imgID):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems/'+imgID
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(CREDS.token)
    }
    res = requests.request("GET", url, headers=headers)
    return res.json()

"""
    Download image and place in WD
"""
def downloadIMG(url, file_name='imgToRecognize.jpg'):
    downloadResponse = requests.get(url)
    destination_folder = './downloads/'
    with open(os.path.join(destination_folder, file_name), 'wb') as f:
        f.write(downloadResponse.content)
        f.close()

"""
    Request the CompreFace API to recognize faces
"""
def recognizeFace(url):
    downloadIMG(url)
    headers = {
        'x-api-key': '0bedc62b-b2a4-4eb2-8efd-b62cc275e23c',
    }

    files = {
        'file': open('./downloads/imgToRecognize.jpg', 'rb'),
    }

    res = requests.post('http://localhost:8000/api/v1/recognition/recognize?face_plugins=landmarks, gender, age', headers=headers, files=files)
    return res.json()

PHONES_TO_PERSON = {
    'BE2026': 'chimu',
    'SM-G970U':'shirleyWhirley',
    'iPhone 11': 'jiusus',
    'Pixel 3': 'bugBoy',
    'Pixel 5a': 'bugBoy',
    'iPhone 12': 'girlBoss',
    'iPhone X': 'me',
    'iPhone 8': 'girlBoss',
    'moto g(7) plus': 'chimu',
    'A6013': 'yuppie',
    'ONEPLUS A6013': 'yuppie',
    'iphone 7': 'dumbestKid',
    'Canon EOS R6': 'other'
}

## Build matrices

Build the takerSubject, picturedWith, and month-based matrices. The values of each cell is a string of comma separated item IDs which we'll hopefully use to request pictures in our final visualizations

In [40]:
import pandas as pd
import numpy as np

NAMES = ['me', 'girlBoss', 'bugBoy', 'jiusus', 'chimu', 'shirleyWhirley', 'yuppie', 'dumbestKid', 'emily', 'other']
RECOGNITION_THRESHOLD = .8 ## The similarity above which we allow a recognition

def getPictureTaker(GPRes):
    mediaMetadata = GPRes['mediaMetadata']
    if 'photo' in mediaMetadata:
        photo = mediaMetadata['photo']
        if 'cameraModel' in photo:
            phoneType = photo['cameraModel']
            try:
                return PHONES_TO_PERSON[phoneType]
            except:
                print('!!!! WARNING: Unrecognized camera')
                print(GPRes)
                downloadIMG(GPRes['baseUrl'], 'pictureTakerErr.jpg')
        else:
            return 'jiusus'
    else:
        return 'video'

"""
    Identify the faces in an image. Given the picture taker, increment the edge between
    pictureTaker and the face in the image.
    
"""
def processRecognition(res, pictureTaker, imgID, month=None):
    if 'result' not in res:
        return
    results = res['result']
    subjects = []
    for result in results: ## Iterates through every face in picture
        possibleSubjects = result['subjects']
        if len(possibleSubjects) == 0:
            continue
        else:
            if possibleSubjects[0]['similarity'] < RECOGNITION_THRESHOLD:
                continue
            else:
                photoSubject = possibleSubjects[0]['subject']
                
                if photoSubject not in NAMES:
                    photoSubject = 'other'
                subjects.append(photoSubject)
                takerSubjectMatrix.at[pictureTaker, photoSubject] += imgID+','
                if month is not None:
                    pictureOfSubjectByMonth.at[photoSubject, month] += imgID+','
                else:
                    subjectCounts[photoSubject]['asSubject'] += 1
                    subjectCounts[pictureTaker]['asPhototaker'] += 1
    subject_i, subject_j = 0, 1
    while subject_i < len(subjects):
        firstSubject = subjects[subject_i]
        while subject_j < len(subjects):
            secondSubject = subjects[subject_j]
            picturedWithMatrix.at[firstSubject, secondSubject] += imgID+','
            picturedWithMatrix.at[secondSubject, firstSubject] += imgID+',' ## Make matrix symmetric for convenience
            subject_j += 1
        subject_i += 1
        subject_j = subject_i + 1

def createSubjectMatrices():
    print('Building subject-taker and photographed with matrices...')
    idFile = open('idFiles/picIDs.txt', 'r')
  
    for imgID in idFile.readlines():
        GPRes = requestIMG(imgID[:-1]) # Cut out the EOL token
        pictureTaker = getPictureTaker(GPRes)
        if pictureTaker == 'video':
            continue
        else:
            try:
                recognitionRes = recognizeFace(GPRes['baseUrl'])
                processRecognition(recognitionRes, pictureTaker, imgID)
            except Exception as e:
                print("!!!! WARNING: recognition api call failure in subject matrices creation !!!!")
                print(e)
                print(recognitionRes)
                
    idFile.close()
    print('Matrices built!')
    
def createMonthMatrices():
    print('Building monthly matrices...')
    for monthEntry in MONTHS:
        print(monthEntry['name'])
        idFile = open('idFiles/months/'+monthEntry['name']+'PicIDs.txt', 'r')
        for imgID in idFile.readlines():
            imgID = imgID[:-1]
            GPRes = requestIMG(imgID) # Cut out the EOL token
            pictureTaker = getPictureTaker(GPRes)
            if pictureTaker == 'video':
                 continue
            pictureBySubjectByMonth.at[pictureTaker, monthEntry['name']]+=imgID+','
            try:
                recognitionRes = recognizeFace(GPRes['baseUrl'])
                processRecognition(recognitionRes, pictureTaker, imgID, monthEntry['name'])
            except Exception as e:
                print("!!!! WARNING: recognition api call failure in month matrix creation !!!!")
                print(e)
                print(recognitionRes)
        idFile.close()
    print('Matrices built!')
        
        
def createCategoryMatrix():
    print('Building category matrix...')
    for category in CATEGORIES:
        idFile = open('idFiles/categories/'+category[-1]+'PicIDs.txt', 'r')
        for imgID in idFile.readlines():
            imgID = imgID[:-1] # Cut out the EOL token
            GPRes = requestIMG(imgID) 
            pictureTaker = getPictureTaker(GPRes)
            if pictureTaker == 'video':
                 continue
            subjectCategory.at[pictureTaker, category[-1]]+=imgID+','
        idFile.close()
    print('Matrix built!')

Run all our cells to populate the matrices for the frontend

## Populate the subject matrices

In [22]:
init_gp_server()

statDict = {
    'asSubject': 0,
    'asPhototaker': 0
}

subjectCounts = {
    'chimu':dict(statDict),
    'shirleyWhirley': dict(statDict),
    'jiusus':dict(statDict),
    'me':dict(statDict),
    'girlBoss':dict(statDict),
    'bugBoy':dict(statDict),
    'yuppie':dict(statDict),
    'emily':dict(statDict),
    'dumbestKid':dict(statDict)
}



takerSubjectMatrix = pd.DataFrame('', index=NAMES, columns=NAMES)
takerSubjectMatrix.index.name = 'client'
picturedWithMatrix = pd.DataFrame('', index=NAMES, columns=NAMES)
picturedWithMatrix.index.name = 'client'
get_hd_people_pics()
createSubjectMatrices()

Download complete!
Building subject-taker and photographed with matrices...
Matrices built!


## Populate the month defined matrices

In [47]:
init_gp_server()
MONTHS = [{'month':3, 'year':2022, 'name':'March'}]
createMonthMatrices()

Building monthly matrices...
March
Matrices built!


In [28]:
init_gp_server()
pictureBySubjectByMonth = pd.DataFrame('', index=NAMES, columns=[monthEntry['name'] for monthEntry in MONTHS])
pictureBySubjectByMonth.index.name = 'photoTaker'
pictureOfSubjectByMonth = pd.DataFrame('', index=NAMES, columns=[monthEntry['name'] for monthEntry in MONTHS])
pictureOfSubjectByMonth.index.name = 'subject'
get_all_month_pics()
createMonthMatrices()

Download complete!
Building monthly matrices...
August
September
October
November
December
January
February
!!!! WARNING: Unrecognized camera


KeyError: None

## Populate the category matrix

In [ ]:
init_gp_server()
subjectCategory = pd.DataFrame('', index=NAMES, columns = [category[-1] for category in CATEGORIES])
get_all_category_pics()
createCategoryMatrix()




In [18]:
takerSubjectMatrix

,me,girlBoss,bugBoy,jiusus,chimu,shirleyWhirley,yuppie,dumbestKid,emily,other
photoTaker,,,,,,,,,,
me,,AMyo5r1atcDHDQ_z7EUqCrWT4ltJHO-2gQ66OofzfrlJMJ...,AMyo5r1atcDHDQ_z7EUqCrWT4ltJHO-2gQ66OofzfrlJMJ...,AMyo5r1atcDHDQ_z7EUqCrWT4ltJHO-2gQ66OofzfrlJMJ...,,AMyo5r1atcDHDQ_z7EUqCrWT4ltJHO-2gQ66OofzfrlJMJ...,,AMyo5r1atcDHDQ_z7EUqCrWT4ltJHO-2gQ66OofzfrlJMJ...,,
girlBoss,,,,,,,,,,
bugBoy,,,,,,,,,,
jiusus,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,,AMyo5r20t6UlFkdekG85UjykyQQ2FybGOOgOmNvQCeQAGL...,,
chimu,,,,,,,,,,
shirleyWhirley,,,,,,,,,,
yuppie,,,,,,,,,,
dumbestKid,,,,,,,,,,
emily,,,,,,,,,,


Use the response of the API to write the results and required metadata into a data frame:

And write the dataframes to csvs

In [20]:
takerSubjectMatrix.to_csv("data/takerSubject.csv")
picturedWithMatrix.to_csv("data/picturedWith.csv")
pictureBySubjectByMonth.to_csv("data/pictureBySubjectByMonth.csv")
pictureOfSubjectByMonth.to_csv("data/pictureOfSubjectByMonth.csv")
subjectCategory.to_csv("data/subjectCategory.csv")

## Overall stats

In [34]:
### Total pictures 
try:
    res = requests.request("GET", GET_LIBRARIES_URL, headers=PHOTOS_HEADERS)
    res.json()
    albumID = res.json()['albums'][1]['id']
except:
    print('!!!! WARNING: Library request error !!!!') 
    print(res)

res = requests.request("GET", GET_LIBRARIES_URL+'/'+albumID, headers=PHOTOS_HEADERS)
print(res)
res = res.json()
totalPictures = int(res['mediaItemsCount'])



<Response [200]>
{'id': 'AMyo5r3D1LLvf_kSbibpaUvWAbJR2KAASBPIKuiucyboAuyzNRRsg78IoGLkL7OUUpmunXIaB9rkQoi7YzAdWuL2pqZzQR7HRw', 'title': '4844 Hilledwight', 'productUrl': 'https://photos.google.com/lr/album/AMyo5r3D1LLvf_kSbibpaUvWAbJR2KAASBPIKuiucyboAuyzNRRsg78IoGLkL7OUUpmunXIaB9rkQoi7YzAdWuL2pqZzQR7HRw', 'mediaItemsCount': '2738', 'coverPhotoBaseUrl': 'https://lh3.googleusercontent.com/lr/AFz2ejQGr-Bu6ffFX0P7O5BZACvyIEGAfag5mjjTEvRn5V8WI-I7AtPMgiJaRa-lLZG-7AHWei0yyZZb7v8ZXFCf4nqpobWQspCLuwWzp8JxdXjYNu3pL38VcKAb8FDNtXAxrXbKavySyzf2HutOEchb9Vh-QNqysr1a-NA8jgLZ7aueByf8eWogKYYZeWtVu3wxOYyUMLjzMyJJ92uR7pIZ9djt9etb7a_qAJTWzqZTyJQosGVoDsRMJWUBGaKUObLDTNJNKHFh5w-64udGIycIU7V-WCFh_j1mDCRSwHIDO44eh6YoD4Bxq1AefqdLm45aeH8UrahjWVd-c4GmZmcjC4KhVDsDSM0uFUIDA7-DvklPcrGm2TEnKhzLesFMuB3syQ4JmXLUsNK8dUqkDKko3F1vqb9rlvbP0nGcrq1vT5Qwe_XjtI-w00C-y71xdmweHdf1s8GNbwGAPZne_JKd0wfHwlpG4GHeNbeX5ppJSDTjPLCTWNq5LNE0qAYK15oqWyh0I5PmkmZLYf23KZ7MAOSdstsQgLP0aZEbAByUxkNFOQBDCk6VG4Zt0h2ar0r--SHqhR0-mA1QJDvNxtYXvEHx-p9w